In [ ]:
!pip install anthropic


In [7]:
import os
import json
import time
import re
from pathlib import Path
import anthropic
from anthropic import Anthropic

# === SETUP ===
client = Anthropic(api_key="myKey")  # Replace with your real API key

# Folder setup
prompts_dir = Path.home() / "Desktop" / "processed_subs" / "prompts"
output_dir = Path.home() / "Desktop" / "processed_subs" / "scored_prompts_claude"
output_dir.mkdir(parents=True, exist_ok=True)

model_name = "claude-3-haiku-20240307"

# === Claude scoring function ===
def score_prompt(prompt_text):
    try:
        response = client.messages.create(
            model=model_name,
            max_tokens=300,
            temperature=0.2,
            messages=[
                {
                    "role": "user",
                    "content": prompt_text
                }
            ]
        )

        content = response.content[0].text.strip()

        print(f"🧪 Raw response:\n{content[:300]}...\n")  # Preview for debug

        # Try to extract a JSON block using regex
        match = re.search(r"\{[^}]*\"universalism\"[^}]*\}", content, re.DOTALL)
        if match:
            json_str = match.group(0)
            return json.loads(json_str)
        else:
            print("⚠️ No valid JSON block found.")
            return None

    except json.JSONDecodeError as e:
        print(f"⚠️ JSON decode error: {e}")
        return None
    except Exception as e:
        print(f"⚠️ Claude API error: {e}")
        return None


In [11]:
import json
import time
from pathlib import Path

# === Paths ===
prompts_dir = Path.home() / "Desktop" / "processed_subs" / "prompts_claude"
output_dir = Path.home() / "Desktop" / "processed_subs" / "scored_prompts_claude"
output_dir.mkdir(parents=True, exist_ok=True)

# === Settings ===
tokens_per_prompt_estimate = 500  # adjust if needed
tokens_per_response_estimate = 50
delay = 1.2

# === Get all prompts to process ===
all_prompt_files = list(prompts_dir.glob("*.json"))
total_files = len(all_prompt_files)
scored_count = 0

for idx, prompt_file in enumerate(all_prompt_files):
    movie_id = prompt_file.stem.replace("_prompts", "")
    output_path = output_dir / f"{movie_id}_scored.json"

    if output_path.exists():
        scored_count += 1
        continue  # Skip already scored movies

    # === Progress Tracker ===
    est_input_tokens = scored_count * tokens_per_prompt_estimate
    est_output_tokens = scored_count * tokens_per_response_estimate
    percent_done = (scored_count / total_files) * 100

    print(f"\n--- Progress ---")
    print(f"🎬 {scored_count}/{total_files} movies scored ({percent_done:.1f}%)")
    print(f"🧮 Est. input tokens used: {est_input_tokens:,}")
    print(f"🧾 Est. output tokens used: {est_output_tokens:,}\n")

    # === Load prompts ===
    with open(prompt_file, "r", encoding="utf-8") as f:
        prompts = json.load(f)

    scored_chunks = []
    for p in prompts:
        chunk_id = p["chunk_id"]
        prompt = p["prompt"]

        result = score_prompt(prompt)
        if result:
            scored_chunks.append({
                "chunk_id": chunk_id,
                "universalism": result.get("universalism"),
                "egalitarianism": result.get("egalitarianism"),
                "progress": result.get("progress")
            })
        else:
            print(f"❌ Failed to score chunk {chunk_id} in {movie_id}")

        time.sleep(delay)  # Respect Claude rate limits

    # === Save results ===
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(scored_chunks, f, indent=2)

    print(f"✅ Scored with Claude: {movie_id}")
    scored_count += 1



--- Progress ---
🎬 0/443 movies scored (0.0%)
🧮 Est. input tokens used: 0
🧾 Est. output tokens used: 0

🧪 Raw response:
{
  "universalism": -0.5,
  "egalitarianism": -0.3,
  "progress": 0.2
}...

🧪 Raw response:
{
  "universalism": -0.5,
  "egalitarianism": -0.7,
  "progress": -0.3
}...

🧪 Raw response:
{
  "universalism": 0.2,
  "egalitarianism": 0.4,
  "progress": 0.6
}...

🧪 Raw response:
{
  "universalism": -0.8,
  "egalitarianism": -0.9,
  "progress": -0.7
}...

🧪 Raw response:
{
  "universalism": -0.5,
  "egalitarianism": -0.7,
  "progress": -0.8
}...

🧪 Raw response:
{
  "universalism": -0.5,
  "egalitarianism": -0.7,
  "progress": -0.3
}...

🧪 Raw response:
{
  "universalism": -0.5,
  "egalitarianism": -0.7,
  "progress": -0.3
}...

🧪 Raw response:
{
  "universalism": -0.5,
  "egalitarianism": -0.7,
  "progress": -0.3
}...

🧪 Raw response:
{
  "universalism": -1,
  "egalitarianism": -1,
  "progress": -1
}...

✅ Scored with Claude: 1977_Looking.For.Mr.Goodbar.ENG.PAL.en

--- Pr